# Notebook to evaluate and run a pre-trained model from ML TC detection & tracking

First import relevant libraries to run the inference and define common info

In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')

sys.path.append('../resources/library')
from tropical_cyclone.inference import SingleModelInference, get_observations, get_observed_tracks
from tropical_cyclone.mlflow import load_model_from_mlflow, set_mlflow_endpoint, check_backend
from tropical_cyclone.visualize import plot_detections, plot_tracks, plot_track_durations, plot_pod_and_far
from tropical_cyclone.cyclone import compute_pod_and_far
from tropical_cyclone.models import *

In [ ]:
main_dir = '/home/jovyan/work/ml-tropical-cyclones-detection/'

# path to dataset directory (if CMIP6 data must be in the proper grid format)
dataset_dir = f'{main_dir}/data/dataset'
# path to IBTrACS file to match ML model detections
ibtracs_src = f'{main_dir}/data/ibtracs/filtered/ibtracs_main-tracks_6h_1980-2021_TS-NR-ET-MX-SS-DS.csv'
# path to configutation file for the model
config_file = f'{main_dir}src/config/cnns.toml'

# define lat and lon ranges
lat_range = (0,70)
lon_range = (100,280)

Select the model by specfying the run name from the MLFlow and download model, scaler and provenance document

In [ ]:
set_mlflow_endpoint(config_file)
run_name=input()
registered_model, path = load_model_from_mlflow(run_name, scaler=False, provenance=False)
registered_model 

Load the ML model

In [ ]:
device = check_backend()
inference = SingleModelInference(model=registered_model, config_file=config_file, device=device)

## Inference workflow on historical data

Let's get the data on a given time frame (year and month) for the evaluation

In [ ]:
import ipywidgets as widgets

month = widgets.Dropdown(
    options=[('Jan', '01'), ('Feb', '02'), ('Mar', '03'),
             ('Apr', '04'), ('May', '05'), ('Jun', '06'),
             ('Jul', '07'), ('Aug', '08'), ('Sep', '09'),
             ('Oct', '10'), ('Nov', '11'), ('Dec', '12'),
             ('ALL', None)
            ], value='08', description='Month:', disabled=False,)

year = widgets.IntSlider(
    value=2014, min=1980, max=2021, step=1,
    description='Year:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

display(widgets.HBox([year, month]))

In [ ]:
ds, dates = inference.load_dataset(dataset_dir=dataset_dir, year=year.value, month=month.value)

We can now detect and localize the TC centers with the ML model and load also the observed TCs

In [ ]:
detections = inference.predict(ds, patch_size=40)
observations = get_observations(ibtracs_src=ibtracs_src, dates=dates, lat_range=lat_range, lon_range=lon_range)

Apply the tracking algorithm to link the TC centers and get the different tracks

In [ ]:
det_tracks = inference.tracking(detections, max_distance=400.0, min_track_count=12)
obs_tracks = get_observed_tracks(observations)

### Plot tracks

In [ ]:
plot_tracks(det_tracks, obs_tracks, lat_range, lon_range,
           "Detected tracks for during "+str(year.value)+"-"+str(month.value))

### Compare with observations

In [ ]:
obs_tracks = obs_tracks.rename(columns={'ISO_TIME':'time','LAT':'lat','LON':'lon','TRACK_ID':'track_id'})[['time','lat','lon','track_id']]
det_tracks = det_tracks.rename(columns={'ISO_TIME':'time', 'LAT':'lat', 'LON':'lon', 'WS':'ws', 'TRACK_ID':'track_id', 'HAVERSINE':'haversine'})

# convert longitudes to range [0, 360] format
obs_tracks['lon'] = (obs_tracks['lon'] + 180) % 360 - 180
det_tracks['lon'] = (det_tracks['lon'] + 180) % 360 - 180

mathces, results = compute_pod_and_far(dynamicopy, det_tracks, run_name, obs_tracks, 300, print_results=False)
results['pod'] = results['pod'] * 100
results['far'] = results['far'] * 100

plot_pod_and_far(results, '', None)

### Plot tracks duration

In [ ]:
plot_track_durations(run_name, det_tracks, obs_tracks)

## Inference workflow on projection data

Let's get CMIP6 data on a given time frame (year and month) for the evaluation

In [ ]:
import ipywidgets as widgets

climate_model = widgets.Dropdown(
    options=[('CNRM-CM6-1-HR', 'CNRM-CM6-1-HR'), ('CMCC-CM2-VHR4', 'CMCC-CM2-VHR4'), ('EC-Earth3P-HR', 'EC-Earth3P-HR'),
             ('MPI-ESM1-2-HR', 'MPI-ESM1-2-HR')],
    value = 'CMCC-CM2-VHR4',
    style={'description_width': '120px'}, 
    description='Model', disabled=False,
    layout=widgets.Layout(width='300px'))

year_range = widgets.IntRangeSlider(
    value=[2030, 2035],        # initial range
    min=2015,                 # min value
    max=2050,               # max value
    step=1,                # step size
    description='Year range:',
    style={'description_width': '80px'},
    layout=widgets.Layout(width='400px')
)

display(widgets.HBox([climate_model, year_range]))

In [ ]:
ds, dates = inference.load_cmip_dataset_rucio(
    climate_model, 
    'CMIP6', 
    'HighResMIP', 
    'VEGA-DCACHE', 
    start_year=year_range.value[0], end_year=year_range.value[1])

We can now detect and localize the TC centers with the ML model

In [ ]:
detections = inference.predict(ds, patch_size=40)

Apply the tracking algorithm to link the TC centers and get the different tracks

In [ ]:
det_tracks = inference.tracking(detections, max_distance=400.0, min_track_count=12)

### Plot tracks

In [ ]:
plot_tracks(det_tracks, None, lat_range, lon_range, 
            "Detected tracks for "+climate_model.value+" during "+str(year_range.value[0])+"-"+str(year_range.value[1]))